In [ ]:
import os

In [ ]:
os.environ["PINECONE_API_KEY"] = "YOUR KEY HERE"  ## Replace with yours
os.environ["PINECONE_ENVIRONMENT"] = "key here"  ## Default
os.environ["PINECONE_INDEX"] = "cml-default" ## Default